<img src="Images/Logo.png" alt="Logo NSI" style="float:right">

<h1 style="text-align:center">Chapitre 24 : Sécurisation des communications</h1>

Nous savons déjà exactement ce qu'il se passe lorsque l'on navigue vers un site web, par exernple [`https://interstices.info/`](https://interstices.info/).
1. L'URL du site est décodée par le navigateur.
2. Ce dernier isole le protocole (HTTPS), le nom de domaine (`interstices.info`) et le chemin vers la ressource(ici `/`, la *racine* du site).
3. Le navigateur effectue une résolution de nom pour déterminer l'adresse IP correspondant au nom de domaine (`128.93.162.59`).
4. Le navigateur peut alors établir une connexion TCP vers l'adresse IP du serveur web, sur le port `443`.
5. Une fois la connexion établie, client et serveur échangent des données en utilisant le protocole HTTPS.

On se souvient aussi que les communications sur Internet utilisent un ensemble de protocoles, organisés en couches :
1. Couche matérielle avec des protocoles tels que **Ethernet** ou **802.11 N**.
2. Couche Internet, avec le protocole IP permettant de définir des routes, c'est-à-dire l'ensemble des machines du réseau traversées pour atteindre la machine de destination.
3. Couche de transport avec les protocoles UDP ou TCP, qui s'occupent en particulier de garantir l'intégrité des données transmises (garanties minimales pour UDP ou très fortes pour TCP).
4. Couche d'application dans laquelle se trouvent les protocoles de haut niveau : HTTP, IMAP, etc.

Ce processus a été très peu modifié depuis la conception de TCP /IP à la fin des années 1970. Chaque protocole (SMTP, FTP, puis HTTP au milieu des années 1990), s'est inséré dans ce cadre au niveau de la couche d'application.

Cependant, avec la démocratisation d'Internet, du Web et la diversification des usages, des problèmes sont apparus.  
En effet, comme on le sait, les données transmises par le protocole de la couche d'application sont découpés en paquets TCP, eux-mêmes encapsulés dans des paquets IP.  
Ces paquets IP sont envoyés par la source au prochain routeur de son sous-réseau.  
Ce routeur retransmet ensuite le paquet au routeur suivant et ainsi de suite jusqu'à l'arrivée à destination. Chaque routeur peut donc inspecter les paquets pour en connaître le contenu. Cette situation n'est clairement pas idéale. En effet, si l'on utilise un site web pour effectuer des transactions bancaires, renseigner des informations personnelles (impôts, arrêt maladie, etc.), ou simplement exprimer son opinion, on souhaite que le contenu des messages envoyés ne soit connu que de deux entités : la source et la destination.

Ce simple constat nous permet de mettre en avant trois aspects liés à la sécurisation des communications :
* Comment chiffrer le contenu des communications, afin qu'elles ne soient lisible que par la source et la destination?
* Comment garantir que le serveur auquel on se connecte est bien celui de la personne et ou de l'entité auquel on pense se connecter?
* Comment garantir les deux propriétés ci-dessus en réutilisant l'infrastructure d'Internet, à savoir les communications TCP /IP?

### Termes liés à la représentation de l'information.
* **Coder** : représenter de l'information par un ensemble de signes prédéfinis.  
On utilise parfois le verbe **encoder**.
* **Décoder** : interpréter un ensemble de signes pour extraire l'information qu'ils représentent.
* **Chiffrer** : rendre une suite de symboles incompréhensible au moyen d'une **clé de chiffrement**.
* **Déchiffrer/décrypter** : retrouver la suite de symboles originale à partir du message chiffré.  
On utilise le terme **déchiffrer** lorsque l'on utilise une **clé de déchiffrement** pour récupérer le texte initial et le terme **décrypter** lorsque l'on arrive à déterminer le message original sans utiliser la clé.

Coder et décoder s'utilisent lorsqu'il n'y a pas de secret.  
Par exemple, on parle de *codage en complément à deux* des entiers. C'est juste une façon de représenter les entiers positifs ou négatifs par une suite de bits. N'importe qui peut décoder la suite de bits pour déterminer l'entier.  
Le terme *coder* (qui est un anglicisme, *to code*) est aussi utilisé de façon informelle. comme synonyme de *programmer* comme dans la phrase "j'ai codé cette application".  
L'anglicisme *crypter* est à proscrire; on utilisera *chiffrer*.

## Cryptographie symétrique
Une première technique lorsque l'on souhaite chiffrer un message est d'utiliser une méthode de **chiffrement symétrique**.  
Formellement, une telle méthode est donnée par deux fonctions :
* $c(m, k)$ est la fonction de chiffrement.  
Elle prend en arguments un **message en clair** $m$ (une chaîne de caractères par exemple) et une **clé de chiffrement** $k$ (qui peut être une chaîne de caractères, un nombre, etc.).  
Elle produit en sortie une chaîne de caractères chiffrée $s$.
* $d(s, k)$ est la fonction de déchiffrement.  
Elle prend en arguments un **message chiffré** $s$ et une **clé de déchiffrement** $k$ et renvoie le message en clair $m$.

Le terme **symétrique** vient du fait que la **même clé** est utilisée pour chiffrer et déchiffrer le message.  

Un exemple simple de méthode de chiffrement symétrique est le **chiffrement par décalage**, encore appelé codage de César (de l'empereur romain Jules César qui utilisait cette technique pour ses correspondances militaires).  
La méthode consiste à choisir un entier $n$ et à décaler chaque lettre du message initial de $n$ lettres dans l'alphabet (en recommençant à `A` si le décalage fait dépasser `Z`).  
Ici, l'entier $n$ constitue la clé de chiffrement.  
Par exemple, en utilisant un décalage de $5$ lettres, le message 

```
L INFORMATIQUE C EST SUPER
```

devient 

```
Q NSKTWRFYNVZJ H JXY XZUJW
```

La fonction de déchiffrement consiste à prendre un message chiffré et à décaler chaque lettre de $n$ positions vers l'arrière.

Une méthode de chiffrement un peu moins naïve est le chiffrement par XOR (ou *ou exclusif*).  
Celle-ci repose sur l'utilisation de l'opérateur binaire ou exclusif noté $⊕$.  
Étant donné un message $m$ (par exemple `L'INFORMATIQUE C'EST SUPER`) et une clé de chiffrement $k$ (par exemple `NSI`), on recopie plusieurs fois la clé de façon à obtenir une chaîne de la même longueur que le message :

```
L'INFORMATIQUE C'EST SUPER
NSINSINSINSINSINSINSINSINS
```

Chaque caractère du message et de la clé augmentée est ensuite converti en nombre (par exemple en son code Unicode) :

```
76 39 73 78 70 79 82 77 65 84 73 81 85 69 32 67 39 69 83 84 32 83 85 80 69 82
78 83 73 78 83 73 78 83 73 78 83 73 78 83 73 78 83 73 78 83 73 78 83 73 78 83
```

On effectue ensuite l'opération $⊕$ entre chaque nombre du message et de la clé.  
Nous la détaillons ci-dessous entre les nombres 84 (lettre `T`) et 78 (lettre `N`) :

|   |   |   |   |   |   |   |   |   |   |    |
|---|---|---|---|---|---|---|---|---|---|----|
|   | 0 | 1 | 0 | 1 | 0 | 1 | 0 | 0 |   | 84 |
| ⊕ | 0 | 1 | 0 | 0 | 1 | 1 | 1 | 0 |   | 78 |
| _ | _ | _ | _ | _ | _ | _ | _ | _ | _ | _  |
|   | 0 | 0 | 0 | 1 | 1 | 0 | 1 | 0 |   | 26 |

On rappelle que l'opération $⊕$ entre deux bits renvoie `0` si les deux bits sont égaux et `1` s'ils sont différents.  
Le message chiffré sera donc:

```
2 116 0 0 21 6 28 30 8 26 26 24 27 22 105 13 116 12 29 7 105 29 6 25 11 1
```

Une propriété intéressante de l'opérateur $⊕$ est qu'il est réversible si $A ⊕ B = C$, alors $A ⊕ C = B$ et $B ⊕ C = A$ (ce n'est pas le cas des opérateurs logiques *et* et *ou* bits à bits). On peut facilement contrôler que 8$4 ⊕ 78 = 26$, mais aussi que $84 ⊕ 26 = 78$ et $78 ⊕ 26 = 84$.  
Par conséquent, le message chiffré ci-dessus peut être déchiffré en réexécutant l'opération $⊕$ nombre à nombre avec les entiers formant la clé étendue `NSINSINSINSINSINSINSINSINS`.  
De plus, l'opérateur $⊕$ est l'une des opérations de base implémentée par le matériel dans l'unité arithmétique et logique du microprocesseur.  
Ces caractéristiques font que l'opérateur $⊕$ est une brique de base couramment utilisée dans les algorithmes de chiffrement modernes (même si, utilisé naïvement comme nous l'avons fait ci-dessus, il n'apporte pas une grande sécurité).  
Attention cependant: si la méthode de chiffrement est connue, alors une clé trop courte (comme celle choisie ici) peut compromettre la sécurité de la communication.

Parmi les algorithmes de chiffrement symétriques les plus utilisés, on peut citer **AES** (pour l'anglais *Advanced Encryption Standard*, standard de chiffrement avancé) et **ChaCha20**.  
Bien que très complexes, ces deux algorithmes reposent sur des principes similaires au chiffrement XOR décrit plus haut :
* une clé initiale est étendue (mais pas aussi naïvement qu'en la répétant)
* la clé et le message sont mélangés (en utilisant entre autres des opérations $⊕$), de façon réversible.

En plus d'être sûrs (il est excessivement difficile, sans connaître la clé, de décrypter un message), ils sont très efficaces et permettent de chiffrer de longs messages en temps réel.  
Par exemple, ils permettent de chiffrer les données contenues dans un fichier avant de les écrire sur un disque dur, ou de chiffrer des communications audio ou vidéo en temps réel sans impact perceptible sur les performances du système.

### Cryptanalyse. 
La **cryptanalyse** est un ensemble de techniques utilisées pour décrypter un texte, c'est-à-dire retrouver le texte en clair à partir du texte chiffré sans posséder la clé de déchiffrement.  
Une telle tentative de décryptage s'appelle une **attaque** sur le texte chiffré.  
Il existe diverses techniques, très complexes.
* Par exemple, si on dispose d'un programme permettant de chiffrer un texte en utilisant une clé, on peut essayer de chiffrer plusieurs messages choisis, en faisant varier les messages et la clé pour voir comment le résultat chiffré se comporte. 
* Les attaques **statistiques**, quant à elles, tentent de déterminer le contenu du message en collectant des statistiques (fréquences, nombre ou densité de certaines suites d'octets) dans le message chiffré. 
* Une autre technique, si on dispose de la fonction de déchiffrement mais pas de la clé, consiste à exécuter une **attaque par force brute**, en énumérant et en essayant toutes les clés possibles. Si la clé de chiffrement est trop courte, alors ce type d'attaque basique a de fortes chances de réussir. 
* Les attaques par **canal auxiliaire** tentent de déterminer des informations sur l'algorithme de chiffrement, la clé ou le message en mesurant des phénomènes indirectement liés au processus de chiffrement.  
Par exemple, en mesurant précisément la température de certaines zones du microprocesseur pendant le chiffrement, certains attaquants réussissent à déterminer quelle partie de l'unité arithmétique et logique est utilisée et donc quelles opérations sont utilisées et avec quelle fréquence (multiplications, ou exclusif, additions, etc.).  
En mesurant les temps de chiffrement pour un fichier de taille fixe, un attaquant peut parfois déterminer la taille de la clé initiale, avant qu'elle ne soit étendue.  

En combinant ces techniques, d'autres informations plus importantes encore peuvent être déterminées, comme le nombre de bits à `1` dans la clé.  
À l'inverse, le rôle des concepteurs de méthodes de chiffrement est de les rendre résistantes à de telles attaques.

## Cryptographie asymétrique
Malgré les nombreux avantages du chiffrement symétrique, ce dernier possède un défaut important.  
En effet, si deux personnes veulent établir un canal de communication sûr en utilisant une méthode de chiffrement symétrique, les deux personnes doivent d'abord se mettre d'accord sur la clé à utiliser (et évidemment sur l'algorithme lui-même).  
Or, les deux participants sont justement dans la situation où il ne peuvent pas encore communiquer de façon sûre. Ils vont donc avoir un problème pour se mettre d'accord sur la clé :
* soit ils échangent la clé par un moyen de communication non sûr (par e-mail, par courrier, etc.) mais dans ce cas, un attaquant pourrait alors s'emparer de la clé et compromettre la sûreté des futures communications chiffrées avec cette clé
* soit ils échangent la clé par un moyen *non pratique*, par exemple en stockant une clé générée sur un support de stockage et en se rencontrant physiquement (Par exemple dans un parc, un soir de nuit sans lune, avec une mallette menottée au poignet) pour se donner la clé de chiffrement.

Pour résoudre ce problème, diverses techniques ont été développées dès les années 1970, d'abord de manière privée, en particulier par les services secrets britanniques et américains, puis dans la recherche académique publique.  
Ces techniques reposent sur la cryptographie **asymétrique** encore appelée **cryptographie à clé publique**.  
Nous présentons ici trois méthodes de cryptographie à clé publique, par ordre chronologique de leur découverte. 

### Puzzles de Merkle
Les **puzzles de Merkle** sont l'un des premiers exemples de méthode cryptographique à clé publique.  
Ils ont été proposés, en 1974, par le cryptologue américain Ralphe C. Merkle (1952-).  
Alice et Basile souhaitent **se mettre d'accord** sur une clé secrète, c'est-à-dire une clé de chiffrement pour un algorithme symétrique.  
Ils n'ont à leur disposition qu'un canal de communication non sûr qui peut être espionné à tout moment par Ève (diminutif de l'anglais *eavesdropper*, l'oreille indiscrète ou l'espion).  
Alice et Basile procèdent en trois étapes :

* **Etape 1** : Alice génère un très grand nombre $N$ (par exemple 100000) de messages de la forme :

`identifiant : i, clé : k`

où `i` et `k` sont générés aléatoirement. Par exemple, Alice génère un fichier de 100000 lignes contenant :

```
...
identifiant : 129378, clé : abAZda9h!snasjda
identifiant : 821012, clé : sladljl32#QSdsal
identifiant : 321091, clé : 9Sakns281alSA©31
...
```

* **Etape 2** : Alice va ensuite chiffrer chaque ligne avec un algorithme de chiffrement **symétrique** et une clé **de faible longueur** différente pour chaque ligne. Le but est que chaque ligne soit susceptible d'être décryptée avec une attaque par force brute, par exemple en 10 secondes de calcul.  
Pour l'exemple, nous utilisons la méthode par **ou exclusif** de la section précédente avec des clés formées de quatre lettres majuscules.  
On obtient un fichier :

```
...
40 62 32 43 53 51 35 44 32 52 49 101 123 122 116 119 120 105 114 125 109 122 38 41 130 243 101 127 ...
56 32 49 59 37 45 50 60 48 42 32 117 107 100 108 103  96 116 101 103 126 100 55 57 146 237 116 111 ...
35 38 54 47 62 43 53 40 43 44 39  97 112  98  96 115 123 114 106 112 102  98 48 45 137 235 115 123 ...
...
```

* **Etape 3** : Alice envoie le fichier contenant les lignes chiffrées à Basile.  
Naturellement, Ève peut intercepter ce fichier car il est envoyé par un canal non sécurisé.  
Basile choisit **au hasard** une ligne du fichier, par exemple :

```
56 32 49 59 37 45 50 60 48 42 32 117 107 100 108 103  96 116 101 103 126 100 55 57 146 237 116 111 ...
```
Il énumère toutes les clés possibles de petite taille pour trouver celle qui correspond.  
Il arrive à déchiffrer la ligne

```
identifiant : 821012, clé : sladljl32#QSdsal
```
grâce à la clé `QDTU`.  
Basile renvoie alors à Alice le message en clair `821012`.  
Ève peut aussi intercepter ce message mais ne peut a priori rien en faire.
* **Étape 4** : sur réception du message de Basile, Alice regarde dans son fichier en clair la ligne contenant `identifiant : 821012`.  
Ils peuvent tous les deux utiliser maintenant la clé `sladljl32#QSdsal` pour communiquer.

Le point crucial, ici, est qu'Ève ne peut pas deviner quelle ligne a été choisie par Basile.  
Elle n'a donc pas d'autre choix que d'essayer d'attaquer en force brute les 100000 lignes du fichier, et tester chacune de ces lignes déchiffrées pour tenter de décoder la communication entre Basile et Alice. Si forcer une ligne prend une seconde, alors Ève mettra, au pire, 1 000 000 secondes et, en moyenne, 500 000 secondes pour trouver la bonne clé (soit un peu moins 6 jours).  
Pendant ces six jours, Alice et Basile disposent d'un canal sûr, qu'ils peuvent utiliser pour échanger des informations.

Cette technique a été l'une des premières proposées et repose sur l'asymétrie suivante : choisir et attaquer une ligne est une opération réaliste pour Basile, mais déchiffrer toutes les lignes est coûteux pour Ève.  Ici, si découvrir une ligne coûte $M$ secondes et qu'il y a $N$ lignes, le coût moyen pour Ève est de $\frac{M \times N}{2}$ secondes.  
Bien que novatrice pour l'époque, cette technique n'apporte pas de nos jours des garanties suffisantes.  
Mais elle a été une source d'inspiration directe pour la seconde méthode que nous présentons, **le protocole d'échange de clés Diffie-Hellman**.

### Méthode de Diffie-Hellman
Le protocole d'échange de clés de Diffie-Hellman est une autre méthode permettant à deux participants de convenir d'une clé symétrique partagée en utilisant un canal de communication non sûr.  
Ce protocole a été proposé en 1976 par Bailey W. Diffie (1944-) et Martin Hellman (1945-), deux cryptologues américains.  
Il repose sur certaines fonctions mathématiques dont nous décrivons ici les caractéristiques sans rentrer dans les détails.  
Supposons que nous possédions une fonction mathématique $M$ (pour "mélange") prenant en arguments deux entiers et possédant les propriétés suivantes :
1. La fonction $M$ est connue (on connaît sa formule ou l'algorithme qui permet de la calculer).
2. Si on connaît $M(x, y)$ et $x$ alors il est *difficile* de retrouver $y$.  
Par difficile, on souhaite que la seule façon dont on dispose pour trouver un $y$ tel que $M(x, y)$ ait une valeur donnée est d'essayer tous les entiers $y$ possibles.
3. Pour tous entiers $x$, $y$ et $z$, on a $M(M(x, y), z) = M(M(x, z), y)$.

Une analogie couramment utilisée est celle des pots de peintures. Supposons que l'on ait à notre disposition un très grand nombre de couleurs de peintures différentes. Les entiers $x$, $y$ et $z$ correspondent à trois couleurs différentes, par exemple $x$ est un certain jaune bien particulier, $y$ un bleu et $z$ un rouge.  
La fonction $M$ prend 10 cl de deux couleurs et les mélange (pour obtenir 20 cl de peinture).  
Notre fonction $M$ possède bien les trois propriétés ci-dessus.  
* Nous connaissons $M$ et tout un chacun (muni d'un verre doseur) peut la reproduire (propriété 1).  
* Étant données deux couleurs (par exemple jaune et bleu), si on les mélange on obtient 20 cl d'un vert bien particulier.  
Il semble a priori impossible, en ayant à notre disposition le vert et la référence exacte du jaune, de deviner quel bleu a été utilisé, car pour de la peinture il n'existe pas d'opération de *soustraction* qui permettrait de retirer le jaune du vert (propriété 2).  
* Enfin, si on mélange d'abord le jaune et le bleu, puis le rouge on obtient au final 30 cl d'un certain marron.  
On obtient 30 cl du même marron si on mélange d'abord le jaune et le rouge, puis qu'on ajoute le bleu (propriété 3). 

Une remarque importante est que, même si on dispose d'un mélange vert (jaune et bleu) et orange (jaune et rouge), alors si on réunit ces deux mélange on obtiendra 40 cl d'un marron différent (car la proportion de jaune est plus importante).  
La figure suivante décrit le protocole en utilisant l'analogie des couleurs.  
Les étapes permettant le partage de clés sont les suivantes :

<div style="text-align: center">
   <img border="0" alt="Echange de clés DH" src="Images/asymetrique-1.png" > 
</div>

* **Etape 1** : Alice et Basile se mettent avant tout d'accord sur une couleur de base commune.  
Cette couleur est considérée comme *publique*. En effet, Alice et Basile doivent pour l'instant communiquer via un canal non sécurisé.  
On doit donc faire l'hypothèse qu'un attaquant peut avoir connaissance du contenu des communications.

* **Etape 2** : Alice et Basile choisissent ensuite chacun une couleur qu'ils gardent **secrète**.  
Une fois cette couleur choisie, ils la mélangent à la couleur publique pour obtenir chacun un mélange dont il est impossible d'extraire la couleur secrète.

* **Etape 3** : Ils peuvent ensuite chacun s'envoyer leur mélange respectif.  
Ici, un attaquant pourrait connaître les mélanges, mais grâce aux propriétés de la fonction de mélange $M$, il ne peut pas découvrir la couleur secrète.

* **Etape 4** : Alice et Basile ajoutent au mélange reçu leur couleur secrète.  
Ils disposent donc tous les deux d'un mélange des trois couleurs dans les bonnes proportions.  
Ils peuvent utiliser ce mélange comme clé.

Le protocole d'échange de clés de Diffie-Hellman résout de façon élégante le problème d'échange de clé posé par l'utilisation d'un algorithme de chiffrement symétrique.  
L'utilisation de ce protocole laisse toutefois une dernière faiblesse : il ne permet pas l'**authentification** des participants, i.e., Alice n'a pas la garantie que la personne avec qui elle communique de façon sûre est bien Basile.  
La troisième méthode de crypographie asymétrique que nous présentons, RSA, permet de résoudre ce problème.

### Le système RSA
Le système RSA est un système de chiffrement asymétrique basé sur des paires de clés publiques et privées.  
Son nom est formé des initiales de ces trois inventeurs: Ron Rivest (1947-), cryptologue américain, Adi Shamir (1952-), cryptologue israélien, et Len Adleman (1945-), cryptologue américain.  
Tout comme le protocole de Diffie-Hellman, ce système repose sur des théorèmes mathématiques complexes. Ce système se prête également moins bien à des analogies simples, telle que celle des couleurs. Nous n'en détaillons donc que les grands principes et nous nous concentrons sur son utilisation pour l'authentification des participants à une communication en réseau.  
Nous abordons dans un premier temps la partie chiffrement.

Le système RSA consiste en la mise en place d'une **paire** de clé publique et privée pour chaque participant.  
Nous notons $K_{Alice}^{pub}$ la clé publique d'Alice et $K_{Alice}^{priv}$ sa clé privée.  
La notation $K_{Alice}^{\_}(m)$ signifie, *chiffrer $m$ avec la clé $K$ (publique ou privée) d'Alice*.  
La manière exacte de créer ces clés est complexe. La seule chose à retenir est que les deux clés sont liées.  
Étant donné un message $m$, on a :

$$K_{Alice}^{pub}(K_{Alice}^{priv}(m)) = K_{Alice}^{priv}(K_{Alice}^{pub}(m)) = m$$

ce qui signifie que si on chiffre le message $m$ d'abord avec la clé privée puis qu'on chiffre le résultat avec la clé publique, ou dans l'autre ordre, on obtient le message initial.  
En d'autres termes, l'une des deux clés permet de défaire le chiffrement effectué par l'autre clé.  
Les propriétés mathématiques mises en jeux font aussi qu'il est
* impossible, en connaissant $K_{Alice}^{pub}$, de  deviner $K_{Alice}^{priv}$ 
* impossible, en connaissant uniquement $K_{Alice}^{pub}(m)$ ou $K_{Alice}^{priv}(m)$, de deviner $m$.

Une fois ces propriétés établies, si Basile souhaite envoyer un message secret à Alice, les deux procèdent comme suit :
1. Alice diffuse sa clé publique $K_{Alice}^{pub}$.  
Par exemple, elle la met sur sa page web, elle l'envoie par e-mail, etc.  
Cette clé peut être connue de tous.  
La clé privée est gardée secrète par Alice.
2. Basile chiffre son message $m$ en effectuant $K_{Alice}^{pub}(m)$ et l'envoie à Alice.
3. Alice applique sa clé privée au message chiffré $K_{Alice}^{priv}(K_{Alice}^{pub}(m)) = m$ et obtient ainsi le message en clair.

Comme on le voit, le système RSA permet de chiffrer des messages sans s'être mis d'accord sur une clé de chiffrement symétrique commune.  
Ce système possède cependant un inconvénient de taille : le chiffrement et le déchiffrement nécessitent des calculs très coûteux. Cette complexité fait qu'il n'est pas possible d'utiliser RSA pour chiffrer directement des gros volumes de données ou des flux de communication (par exemple une visioconférence avec envoi de vidéo et de son).

Ce problème peut être réglé en remarquant qu'on peut utiliser RSA dans le même but que le protocole d'échange de clé de Diffie-Hellman.  
Si Basile veut échanger des gros volumes de données de façon sûre avec Alice, il peut d'abord choisir une clé pour un algorithme de chiffrement symétrique. Une telle clé est un fichier de quelques milliers d'octets.  
Il peut ensuite envoyer cette clé à Alice, de manière sûre, en utilisant RSA. Une fois la clé symétrique reçue, un algorithme de chiffrement symétrique efficace (tel que **ChaCha20** ou **AES**) peut être utilisé.  
Un autre avantage de RSA, qui a fait sa popularité, est la possibilité de l'utiliser comme système d'authentification.

## Authentification des participants
### Attaque de l'homme du milieu
Comme nous l'avons vu, le chiffrement des communications permet de se protéger contre l'espionnage des communications, c'est-à-dire l'observation passive par un tiers des messages envoyés entre deux participants.  
Considérons maintenant le scénario suivant :
* Supposons qu'Alice soit cliente de l'établissement bancaire BasileBanque. 
* Les communications entre Alice et BasileBanque étant confidentielles, elles doivent évidemment être chiffrées, mais cela n'est pas suffisant.

Marion (la malveillante) peut procéder à l'attaque suivante : elle envoie un e-mail auformat HTML (la plupart des logiciels utilisent le format HTML pour représenter les e-mails avec *contenu riche* (images, polices différentes, couleurs, etc.) à Alice, lui demandant de se connecter d'urgence sur le site de sa banque.  
Le contenu d'un tel e-mail pourrait être :

```html
<html>
    ...
    <body>
        Veuillez vous connecter d'urgence au site
        <a href="http://marion.com">BasileBanque.com</a>.
    </body>
</html>
```

Ce dernier s'affichera de la façon suivante:

---
<html>
    <body>
        Veuillez vous connecter d'urgence au site
        <a href="http://marion.com">BasileBanque.com</a>.
    </body>
</html>

---

Si Alice n'est pas méfiante, et que Marion utilise en plus d'autres techniques (comme de reproduire exactement la page d'accueil du site [BasileBanque.com](http://marion.com)), alors elle ne se rendra pas compte qu'elle est connectée au mauvais site.  
Marion pourra donc recevoir tous les messages envoyés par Alice.  
Elle pourra aussi les retransmettre au véritable site [BasileBanque.com](http://BasileBanque.com), en se faisant passer pour Alice.  
Marion sera donc dans une position d'intermédiaire entre Alice et Basile.  
Elle pourra retransmettre passivement les requêtes d'Alice à BasileBanque, ou les modifier avant de les retransmettre (par exemple changer le montant et le destinataire d'un virement bancaire), ce que l'on peut illustrer par le schéma suivant:

<div style="text-align: center">
   <img border="0" alt="Attaque de l'homme du milieu" src="Images/authentification-1.png" > 
</div>

<!---
https://viewer.diagrams.net/?tags=%7B%7D&highlight=0000ff&edit=_blank&layers=1&nav=1#R1VZLb5wwEP41HBuxOI%2F2uGySplJz6R6yOVowxa6Mh5ohQH59zWIvOCjbRGqTVkLC32fPeN4QsU3ZfTa8EreYg4qSOO8idhklycckiYYnzvuRYOdsJAoj85FaTcRWPoIjY8c2Moc6OEiIimQVkhlqDRkFHDcG2%2FDYd1ThrRUvYEFsM66W7J3MSXi3Lib%2BBmQh%2FM2r80%2FjTsn9YedJLXiO7YxiVxHbGEQaV2W3ATXEzsdllLt%2BZvdgmAFNLxEQIvvy%2BG3N%2Brt%2Bt0vT%2Bn6jbj6cOtuo9w5Dbv13UKO2r9Rgo3MY1MQWoSGBBWquviJWllxZ8gcQ9S57vCG0lKBSuV3oJO1m6%2FtB1cmZQ5ed07wHvQeaTL%2Bbg5nUACexPfJyNXFD6yHzlsgUr2uZefpaKm%2FS6Pfg7LPhdFSNjcngSAx9WXJTAB05lxySbpsFsARrtpUzoDjJh9AO7sq2OJybMmsXLrmvSLTT%2B8BV425aK2ndepr%2BMNetkATbiu%2F9b22Dh3l1OsEQdMfDuHTbDwjXHX0I26nXLhwlZm3G4r8UpmQRpltuJOp3j9PB5X8kUGdvOjjiVwyOtx8A7H8YAGxR2SmvpYKU658NnGRYvnuNs98PA%2Ft9%2FSNFbuH05d3vzX5f2NUv
-->

Ici, le chiffrement n'est d'aucune aide.  
En effet, même si Alice établit un canal sûr entre elle et Marion, et que Marion en établit un entre elle-même et BasileBanque, l'attaque peut quand même avoir lieu.  
On appelle cela une **attaque de l'homme du milieu** (ou *man in the midle attack* en anglais, souvent abrégé en MITM).  

La faille fondamentale permettant cette attaque est l'absence d'**authentification**.  
Ici, le serveur de Marion, qui imite le comportement du serveur de BasileBanque, n'a pas eu à **prouver** qu'il était bien une machine appartement à BasileBanque.  
Nous allons maintenant voir comment le système RSA peut être utilisé pour **authentifier** les communications.

### Certificats et tiers de confiance
Comme on l'a vu, afin de sécuriser entièrement les communications entre deux participants, il faut non seulement les chiffrer, mais aussi que chaque participant puisse s'assurer de l'identité de son correspondant.  
Avant de rentrer dans les détails de l'authentification des communications en réseau, nous allons faire une analogie avec l'authentification des personnes en France.  
L'État délivre aux citoyens français (qui font les démarches) une carte nationale d'identité.  
Si un bureau de poste souhaite authentifier une personne pour lui permettre de retirer un colis, elle peut vérifier la carte d'identité de la personne.  
La carte d'identité n'est qu'un bout de carton plastifié et ne permet donc pas à elle seule de prouver l'identité d'une personne.  
Le système fonctionne parce que le bureau de poste **fait confiance** à l'État qui a fait les vérifications nécessaires pour s'assurer de l'identité de la personne avant de lui délivrer la carte.

Les communications en réseau sur Internet sont authentifiées de la même façon.  
L'entité qui veut prouver son identité présente un **certificat**.  
Ce dernier doit être délivré par une autre entité, en laquelle les deux participants ont confiance, que l'on appelle un **tiers de confiance**.  
Si Alice, qui souhaite correspondre avec BasileBanque, veut s'assurer que c'est bien Basile qui est le propriétaire du site, elle devra lui demander un certificat.  
Ce dernier sera délivré par un tiers de confiance, Théo (le tiers de confiance).  
Évidemment, dans le cadre de communications en réseau, le certificat doit être lui aussi numérique.  
Ces certificats numériques sont créés à partir des clés RSA publiques et privées des participants.
* **Etape 1** : Basile va voir Théo. Théo vérifie que Basile est bien le propriétaire du site [BasileBanque.com](https://BasileBanque.com).  
Il peut par exemple constater que Basile peut administrer le site, rajouter des pages, ou qu'il peut présenter des documents légaux (preuve d'achat du nom de domaine par exemple).  
Une fois ces vérifications faites, Théo utilise sa clé privée $K_{Theo}^{priv}$ pour chiffrer la clé publique de Basile :
$$s=K_{Theo}^{priv}(K_{Basile}^{pub})$$  
Un tel fichier $s$ s'appelle un **certificat**.  
Dans ce cas particulier, on dit que Théo **signe** la clé publique de $K_{Basile}^{pub}$ avec sa clé privée.
* **Etape 2** : Alice veut se connecter à [BasileBanque.com](https://BasileBanque.com).  
Lors de la connexion, le site lui fournit la clé publique de Basile, $K_{Basile}^{pub}$ ainsi que le certificat $s$.
* **Etape 3** : Alice récupère alors la clé **publique** de Théo, en qui elle a confiance.  
Elle calcule :
$$K_{Theo}^{pub}(s)=K_{Theo}^{pub}(K_{Theo}^{priv}(K_{Basile}^{pub}))=K_{Basile}^{pub}$$  
Alice peut alors comparer la clé publique que Basile lui a fournie et celle résultant du déchiffrement de la signature.  
Si les deux sont les mêmes, Alice a la **garantie** que Basile est bien passé voir Théo et que ce dernier a bien fait toutes les vérifications nécessaires.
* **Etape 4** : Alice ayant authentifié [BasileBanque.com](https://BasileBanque.com) comme étant bien le site détenu par Basile, elle peut initier avec lui une communication sécurisée, par exemple en utilisant la clé publique pour chiffrer une clé symétrique ou en utilisant Diffie-Hellman pour se mettre d'accord sur une clé.

## Le protocole HTTPS
Nous sommes maintenant équipés pour expliquer le fonctionnement du protocole HTTPS (**HyperText Transfer Protocole Secure**, protocole de transfert de documents hypertexte sécurisé).  
Bien que ce dernier repose sur les concepts cryptographiques que nous avons présentés, d'autres considérations ont été prises en compte lors de sa conception :
* **compatibilité avec HTTP** : le but n'est pas de recréer un nouveau protocole, mais juste de rendre HTTP plus sûr
* **modularité et compatibilité future** : les systèmes cryptographiques présentés reposent sur le fait que certaines opérations demandent tellement de temps de calcul à l'attaquant qu'elles sont incassables en pratique.  
Cependant, les capacités de calculs des ordinateurs augmentant avec les progrès technologiques, il faut pouvoir **changer la difficulté** des problèmes à résoudre (par exemple augmenter la taille des clés ou changer les algorithmes de chiffrement) sans remettre en cause tout le protocole
* **performances** : un site web populaire devra effectuer plusieurs milliers d'opérations de chiffrement et déchiffrement par seconde.  
Comme on l'a vu, les méthodes symétriques sont peu coûteuses mais demandent de pouvoir échanger une clé de façon sûre. Les méthodes asymétriques sont sûres mais très coûteuses en temps de calcul, il faut donc limiter l'usage de ces dernières au strict minimum.

### Autorités de certifications et format X.509
Une **autorité de certification** (ou AC) est une entité habilitée à délivrer des certificats.  
Une AC est un **tiers de confiance**.  
Parmi les différentes AC on retrouve :
* des entreprises spécialisées
* des associations à but non lucratif (comme l'initiative [Let's Encrypt](https://letsencrypt.org/), libre et gratuite)
* des états

Comme nous l'avons vu, le rôle des tiers de confiance est d'attester, au moyen d'un certificat numérique, qu'une entité (entreprise, personne, etc.) est bien qui elle prétend être.  
Les autorités de certifications forment un *club* très fermé. En effet, les systèmes d'exploitation et les navigateurs web, en particulier, possèdent les clés publiques des AC sous forme de fichiers. Elles sont en nombre relativement restreint.  
Par exemple, au début 2020, la fondation Mozilla, qui produit le navigateur web libre Firefox, reconnaît environ un peu plus d'une centaine d'autorités de certification. Chacune d'elles répond à des critères très stricts, vérifiés par des audits réguliers dont les résultats sont publics.  
Les sociétés Microsoft, Apple et Google (qui développent des systèmes d'exploitation et des navigateurs) ont elles aussi leurs critères (qui ne sont pas forcément les mêmes que ceux de la fondation Mozilla) pour accepter les autorités de certification.  
Les systèmes d'exploitation et les navigateurs web possèdent une copie de chacune de leurs clés publiques.

Les AC émettent des certificats.  
Ces derniers fonctionnent sur le principe expliqué précédemment, avec quelques considérations techniques supplémentaires. Le format standard de certificat est le format **X.509**.  
Ce dernier est un format de fichier binaire, contenant entre autres :
* l'identifiant de l'AC qui signe le certificat
* l'identifiant de l'entité certifiée
* la date de validité du certificat
* la clé publique de l'entité certifiée
* l'algorithme utilisé pour la signature du certificat
* la signature du certificat par l'AC

Sous Unix, la commande [`openssl`](https://www.unix.com/man-page/posix/1ssl/openssl/) permet d'obtenir des informations sur un certificat.  
Par exemple, le [certificat](Fichiers/interstices.pem) du site [`https://interstices.info`](https://interstices.info) peut être affiché comme ceci :

```
utilisateur@machine:~$ openssl x509 -in interstices.pem -text
```

Ici, le paramètre `x509` indique que l'on travaille avec un certificat au format X.509, le paramètre `-in` permet de donner le fichier de certificat, ici `interstices.pem` et le paramètre `-text` indique que l'on souhaite afficher le contenu du certificat de façon lisible.  
La [sortie](Fichiers/informations_certificat.txt) de la commande (abrégée) est:

```
Certificate:
    Data:
        Version: 3 (0x2)
        Serial Number:
            2b:90:6a:ca:18:ac:59:69:42:5f:4a:1f:fe:fe:7b:f5
        Signature Algorithm: sha384WithRSAEncryption
        Issuer: C = NL, O = GEANT Vereniging, CN = GEANT OV RSA CA 4
        Validity
            Not Before: Sep 15 00:00:00 2021 GMT
            Not After : Sep 15 23:59:59 2022 GMT
        Subject: C = FR, L = Le Chesnay-Rocquencourt, O = INSTITUT NATIONAL DE RECHERCHE EN INFORMATIQUE ET EN AUTOMATIQUE, OU = DSI, CN = www.interstices.info
        Subject Public Key Info:
            Public Key Algorithm: rsaEncryption
                RSA Public-Key: (2048 bit)
                Modulus:
                    00:b6:dc:f4:01:73:45:de:e0:14:15:ae:a9:d0:5b:
                    ...
                Exponent: 65537 (0x10001)
                ...
    Signature Algorithm: sha384WithRSAEncryption
         8a:0e:6f:9b:7d:8f:e9:c0:80:7b:db:ff:24:87:5f:93:60:89:
         ...
```


L'AC ayant produit ce certificat est [GEANT Vereniging](https://wiki.geant.org/display/TCSNT/TCS+wiki+%282020%29+Sectigo).  
Le certificat n'est
valide que sur la période du 15 septembre 2021 au 15 septembre 2022, et devra être renouvelé après.  
Le site certifié est `www.interstices.info`.  
La clé publique du site, c'est-à-dire celle qui sera utilisée par le navigateur web pour chiffrer la communication initiale, fait 2048 bits, et est donnée par les deux champs `Modulus` et `Exponent` (raccourcis pour des raisons d'espace).  
Enfin, l'algorithme de signature est `sha384WithRSAEncryption`. Cet algorithme est une variation de la signature RSA que nous avons présentée dans la section précédente.  
Ici, plutôt que de signer (i. e., de chiffrer) toutes la clés publiques du site, l'AC a d'abord calculé une somme de contrôle avec l'algorithme de hachage SHA384. Ce dernier produit un entier de 384 bits en *mélangeant* les différents octets du certificat (tous les champs présentés, exceptés Signature Algorithm: et la signature). Ce *résumé* du certificat (incluant le nom de l'entité, les dates de validité et la clé publique) est ensuite signé (i.e., chiffré) avec la clé privée de l'autorité *GEANT Vereniging*.  

Lorsque le client, Alice, souhaite vérifier que le site [`https://interstices.info`](https://interstices.info) est authentifié, il lui suffit de retirer les deux derniers champs du certificat, de calculer la somme de contrôle avec l'algorithme SHA384, et de la chiffrer avec [la clé publique de l'autorité GEANT Vereniging](https://crt.sh/?caid=160137).  
Cette dernière étant une autorité reconnue, les navigateurs web possèdent une copie de sa clé publique, afin de procéder aux vérifications.  
Avant d'illustrer comment les certificats sont utilisés dans le cadre du protocole HTTPS, nous terminons notre présentation des autorités de certifications en citant les niveaux de certifications. En effet, il existe trois niveaux de certification possibles :
* **DV** (pour l'anglais *domain validated*, soit **validation du domaine**), c'est le niveau le plus basique.  
L'AC a juste vérifié que la personne qui a demandé le certificat est bien la personne qui contrôle le nom de domaine certifié.  
Ce contrôle peut être fait à distance, en demandant à l'administrateur de déposer à une certaine URL un fichier généré par l'AC (par exemple `https://interstices.info/fichier`).
* **OV** (pour l'anglais *organisation validated*, soit **validation de l'organisation**), est le niveau intermédiaire.  
Des vérifications sont faites sur l'existence légale de la personne morale ou physique correspondant à l'entité certifiée.  
L'AC effectue des vérifications manuelles auprès de l'entité (généralement une entreprise) qui demande la certification.  
Par exemple, l'AC passe un appel téléphonique au siège social de l'entreprise pour s'assurer de son existence, en plus des vérifications faites au niveau DV.
* **EV** (pour l'anglais *extended validated*, soit **validation étendue**), est le niveau le plus haut de certification.  
L'AC vérifie les documents légaux fournis par l'entreprise à certifier et recoupe ces vérifications avec les registres publics (registres des entreprises par exemple), en plus des vérifications faites au niveau OV.

### Détails du protocole HTTPS
Le protocole HTTPS qui permet d'établir des communications sécurisées entre un client et un serveur web est en fait simplement le protocole HTTP, auquel une couche de cryptographie a été ajoutée.  
En effet, HTTPS est simplement la réunion de deux protocoles existants, HTTP, pour l'interaction avec le serveur web et TLS (**Transport Layer Security**, sécurité de la couche de transport). 

Rappelons-nous le fonctionnement du protocole HTTP.  
Lorsqu'un client (par exemple un navigateur web) souhaite afficher la page [https://interstices.info/](https://interstices.info/), il initie une connexion TCP vers l'adresse IP de la machine `interstices.info` sur le port 80.  
Une fois la connexion établie, le navigateur envoie une requête HTTP, c'est-à-dire une suite d'octets de la forme :

```
GET / HTTP/1.1
Host: interstices.info
```

Le protocole TLS ajoute simplement une phase permettant l'authentification du serveur et la mise en place sécurisée d'une clé de chiffrement symétrique, appelée clé de session.  
Une fois cette clé déterminée et connue du client et du serveur, le client et le serveur échangent des requêtes et réponses HTTP (la requête GET précédente), mais ces dernières seront chiffrées avant l'envoi et déchiffrées lors de la réception.  
L'authentification empêche l'exécution d'attaques de l'homme du milieu, et le chiffrement empêche les routeurs et autres machines intermédiaires de lire le contenu des messages.  
La figure suivante décrit la phase de mise en place, appelée **poignée de main TLS** (TLS handshake en anglais). 

<div style="text-align: center">
   <img border="0" alt="Mise en place d'une connexion HTTPS" src="Images/TLS-1.png" > 
</div>


Cette phase initiale n'étant pas compatible avec le protocole HTTP, un port différent a été choisi pour le protocole HTTPS, à savoir le port 443.  
Les étapes de cette poignée de main reprennent chacun des points développés tout au cours du chapitre.
* Étape 1 : Le client envoi un message initial (nommé "Hello") ainsi que des options.  
En particulier, le client indique les différents algorithmes cryptographiques qu'il peut utiliser, ainsi que d'autres paramètres techniques.
* Etape 2 : Le serveur envoie sa réponse, contenant entre autres le certificat X.509 contenant sa clé publique, signée par une autorité de certification.
* Etape 3 : Le client vérifie le certificat au moyen de la clé publique de l'AC (qu'il doit posséder).  
Il procède aussi à d'autres vérifications (comme le fait que le certificat est toujours valide).
* Etape 4 : Le client et le serveur conviennent d'une clé de session pour un algorithme symétrique.  
Ils peuvent soit choisir de chiffrer une clé choisie par le client avec la clé publique du serveur, soit utiliser le protocole d'échange de clé de Diffie-Hellman pour convenir d'une clé de session partagée.
* Etape 5 : Le serveur est authentifié par le client, et les deux ont convenu d'une clé de session.  
Ils peuvent donc échanger de manière sûre des messages du protocole HTTP en les chiffrant.

Tous ces aspects sont observables en inspectant l'icône de cadenas se trouvant dans la barre d'adresse des navigateurs web.  
Par exemple, pour le navigateur Firefox, connecté au site [https://interstices.info/](https://interstices.info/) on peut afficher les propriétés de la connexion, comme indiqué à la figure suivante :

<div style="text-align: center">
   <img border="0" alt="Les propriétés d'une connexion HTTPS" src="Images/TLS-2.png" > 
</div>

Dans cette figure, on peut voir entre autres détails que la connexion utilise les options `TLS_ECDHE_RSA_WITH_AES_128_GCM_SHA256`, ce qui signifie qu'elle utilise le protocole TLS (plutôt que son prédécesseur SSL, moins sûr).  
Le sigle ECDHE, pour **Elliptic Curve Diffie-Hellman Ephemeral**, indique que la clé de session est échangée par le protocole de Diffie-Hellman dans sa variante utilisant les courbes elliptiques comme problème mathématique sous-jacent.  
L'authentification est assurée par RSA.  
Le chiffrement symétrique est quant à lui assuré par l'algorithme AES128.

Certains problèmes peuvent survenir lors de la poignée de main TLS.  
Le plus courant est l'utilisation par le serveur d'un certificat non signé ou signé par une entité inconnue.  
Un navigateur indique alors un message similaire à celui de la figure suivante :

<div style="text-align: center">
   <img border="0" alt="Connexion non sûre" src="Images/TLS-3.png" > 
</div>

La présence de ce message indique que la communication est chiffrée, mais pas authentifiée.  
Le serveur web peut alors appartenir à un utilisateur malveillant effectuant une attaque de l'homme du milieu.  
En pratique, ce phénomène se produit aussi lorsque le certificat n'est plus valide, par exemple si l'administrateur du site n'a pas renouvelé son certificat à temps.  
En cas d'absence de chiffrement (utilisation du protocole HTTP simple), le navigateur affiche généralement un cadenas rouge ou barré et déconseille la saisie de mots de passe ou autre information sensible.

## Analyseur de paquet
Un analyseur de paquets permet d'observer les échanges et les protocoles mis en place lors de l'utilisation du réseau internet.  
Cela permet de mieux comprendre les différents échanges lors de la navigation sur le Web.  
Par exemple, voici quelques fichiers que vous pouvez étudier avec le logiciel [Wireshark](https://www.wireshark.org/).
* [échanges de paquets entre un client et le site https://interstices.info](Fichiers/interstices.pcapng)
* [échanges de paquets entre un client et la page https://www.example.com/index.html](Fichiers/https_example.pcapng)

## Exercices
### Exercice 1
Pour chacun des sigles ou noms suivants, le décrire, en une phrase, sans donner la signification du sigle.
1. TLS
2. RSA
3. Diffie-Hellman
4. X.509
5. ChaCha20
6. HTTPS
7. AES

### Exercice 2
Écrire en Python une fonction `chiffre_xor(msg, cle)` qui prend en arguments deux chaînes d'octets (type bytes) et qui renvoie le
chiffrement XOR du message avec la clé, sous forme d'une chaîne d'octets.

On rappelle que l'on peut créer une chaîne d'octets en utilisant le constructeur [`bytes`](https://docs.python.org/fr/3/library/stdtypes.html#bytes) et en lui donnant en argument une liste d'entiers, chacun compris entre 0 et 255.

Indication: On rappelle que pour un chiffrement XOR, la clé doit être *étendue* de façon à avoir la même taille que le message.  
Cependant, il n'est pas nécessaire de créer une chaîne intermédiaire contenant des copies répétées de la clé.  
On pourra en effet faire une utilisation judicieuse de l'opérateur `%`.

### Exercice 3
Comme expliqué dans le cours, un chiffrement XOR simple n'apporte pas une grande sécurité.  
On va montrer qu'en connaissant quelques informations on peut facilement retrouver la clé si cette dernière est trop courte. Soit la chaîne d'octets chiffrée :
```
b'\xOe6/+y;.< x-(7,,\x9b\xfOz48z:646<z*\x9a\xf3(64+<{'
```

On sait que les 4 derniers caractères du message en clair sont `"nse!"`.  
On sait aussi que la clé fait exactement 3 caractères et que ce sont des lettres majuscules sans accent.

Écrire un programme Python, utilisant la fonction `chiffre_xor` de l'exercice précédent, qui essaye toutes les combinaisons de clé jusqu'à trouver la bonne.  
Mesurer le temps d'exécution. On pourra utiliser la fonction `time.perf_counter()`.

### Exercice 4
On se place dans le contexte de la figure suivante :

<div style="text-align: center">
   <img border="0" alt="Mise en place d'une connexion HTTPS" src="Images/TLS-1.png" > 
</div>

Le client est un navigateur web classique (par exemple Firefox).  
Le serveur web est configuré sur une machine dont le nom de domaine est `www.monsite.fr`.  
Dans chacune des situations suivantes, dire quelle étape de la poignée de main TLS échoue.  
Les questions sont indépendantes.
1. Le serveur web n'est pas configuré pour supporter le protocole HTTPS et ne sert des pages qu'en HTTP.
2. Le fichier contenant le certificat côté serveur est périmé.
3. L'utilisateur du navigateur pointe ce dernier vers l'URL `http://www.monsite.fr:443`.
4. L'administrateur du serveur a créé une paire de clé publique et privée, a signé le certificat que le serveur envoie aux clients et effacé les clés.
5. Le navigateur commence à afficher la page de garde du site. Le câble connectant le serveur au réseau est coupé.

### Exercice 5
Télécharger le fichier [cert.pem](Fichiers/cert.pem). 

Utiliser la commande `openssl` pour déterminer les informations de ce certificat.  
Donner en particulier sa date d'expiration et le nom de l'AC qui l'a signé.

### Exercice 6
On rappelle qu'une socket est un objet représentant une connexion TCP vers un hôte et un port donné.  
En Python, un client se connectant en TCP à un serveur peut s'écrire comme suit :

```python
import socket
hote = "www.iana.org"

#Prend en argument un coup te nom et port
s = s.create_connection((hote, 80))
```

Sur un tel objet `s`, on peut utiliser une méthode `s.read(n)` où `n` est un entier strictement positif.  
La méthode renvoie une chaîne d'octets (type bytes) dont la taille peut être inférieure ou égale à `n`. 
Une taille de `0` indique que le serveur a fini sa réponse.  
La méthode `s.write(b)` où `b` est une chaîne d'octets envoie la chaîne au serveur.  

On peut automatiquement rajouter une couche TLS à un programme utilisant une socket en utilisant le module Python [ssl](https://docs.python.org/fr/3/library/ssl.html).

```python
import socket
import ssl

hote = "www.iana.org"
ctx = ssl.create_default_context()

#Prend en argument un coupte nom et port
s = s.create_connection((hote, 443))
ss = ctx.wrap_socket(s, server_hostname = hote)

#On réutitise ss comme une socket ici.
```

Dans le code ci-dessus, l'objet `ss` est une socket, peut s'utiliser comme une socket, qui chiffrera automatiquement les messages envoyés et déchiffrera les messages lus.  
En plus des méthodes `.read()` et `.write()`, l'objet `ss` possède une méthode `.getpeercert()` qui renvoie un dictionnaire décrivant le certificat.

Compléter le code ci-dessus pour qu'il récupère la page de garde du site web [`https://www.iana.org`](https://www.iana.org).  
Une fois ce code écrit et testé, changer le paramètre `server_hostname` pour qu'il indique un hôte différent de celui stocké dans la variable `hote`.  
Expliquez le message d'erreur.

### Exercice 7
On considère le cas d'un site Internet `https://www.monsite.fr`.  
Ce dernier a été mal configuré et le fichier `privkey.pem`, contenant la clé privée correspondant à la clé publique contenue dans le certificat, est téléchargeable.  
Le site choisi d'utiliser RSA pour l'échange de clé de session (et non pas Diffie- Hellman).  

Indiquer ce que peut faire un utilisateur malveillant en possession de cette clé privée.

## Liens :
* Document accompagnement Eduscol : [Sécurisation des communications](https://eduscol.education.fr/document/11480/download)
* [Secure Communications over Insecure Channels](Ressources/Merkle_1978.pdf) - Merkle (1978)
* [New directions in cryptography](Ressources/Diffie-Hellman_1976.pdf) - Diffie-Hellman (1976)
* [A method for obtaining digital signatures and public-key cryptosystems](Ressources/RSA_1978.pdf) - Rivest-Shamir-Adleman (1978)
* RFC 7539 : [ChaCha20 and Poly1305 for IETF Protocols](https://tools.ietf.org/html/rfc7539)
* RFC 3565 : [Use of the Advanced Encryption Standard (AES) Encryption Algorithm in Cryptographic Message Syntax (CMS)](https://tools.ietf.org/html/rfc3565)
* RFC 8017 : [RSA Cryptography Specifications](https://tools.ietf.org/html/rfc8017)
* RFC 8446 : [Transport Layer Security](https://tools.ietf.org/html/rfc8446)
* RFC 2818 : [HTTP Over TLS](https://tools.ietf.org/html/rfc2818)
* RFC 5280 : [Internet X.509 Public Key Infrastructure Certificate](https://tools.ietf.org/html/rfc5280)
* RFC 7935 : [The Profile for Algorithms and Key Sizes for Use in the Resource Public Key Infrastructure](https://tools.ietf.org/html/rfc7935)
* ANSSI : [Mécanismes cryptographiques](https://www.ssi.gouv.fr/administration/guide/mecanismes-cryptographiques/)
* ANSSI : [Recommandations de sécurité relatives à TLS](https://www.ssi.gouv.fr/administration/guide/recommandations-de-securite-relatives-a-tls/)
* ANSSI : [CRYPTO : LE WEBDOC’](https://www.ssi.gouv.fr/administration/guide/crypto-le-webdoc/)
* INRIA : [Livre blanc sur la cybersécurité](https://www.inria.fr/fr/livre-blanc-inria-cybersecurite)
* Interstices : [Vérifier la sécurité de nos communications](https://interstices.info/verifier-la-securite-de-nos-communications/)
* Interstices : [Cryptographie, du chiffre et des lettres](https://interstices.info/cryptographie-du-chiffre-et-des-lettres/)
* Interstices : [Nombres premiers et cryptologie : l’algorithme RSA](https://interstices.info/nombres-premiers-et-cryptologie-lalgorithme-rsa/)
* Interstices : [Animation interactive RSA](https://animations.interstices.info/interstices-rsa/rsa.html)
* Interstices : [À l’attaque des codes secrets](https://interstices.info/a-lattaque-des-codes-secrets/)
* Interstices : [Animation interactive Codes secrets](https://jdolivet.github.io/NSI-Cours/Terminale/Sites/codes-secrets/)
* Société Informatique de France : [Cybersécurité](https://www.societe-informatique-de-france.fr/wp-content/uploads/2017/09/1024_11_2017_25.pdf)
* Société Informatique de France : [Gestion sécurisée de données personnelles](https://www.societe-informatique-de-france.fr/wp-content/uploads/2015/03/1024_5_2015_17.pdf)
* CultureMaths : [Voyage au cœur de la cryptographie](https://culturemath.ens.fr/thematiques/lycee/voyage-au-coeur-de-la-cryptographie)
* CultureMaths : [Cryptographie asymétrique et courbes elliptiques](https://culturemath.ens.fr/thematiques/lycee/cryptographie-asymetrique-et-courbes-elliptiques)
* CultureMaths : [La cryptographie et les ordinateurs quantiques](https://culturemath.ens.fr/thematiques/informatique/la-cryptographie-et-les-ordinateurs-quantiques)
* Images des Mathématiques : [COMMENT LES MATHÉMATIQUES ONT INVESTI LA CRYPTOLOGIE (1)](https://images.math.cnrs.fr/Comment-les-mathematiques-ont-investi-la-cryptologie-1.html)
* Images des Mathématiques : [COMMENT LES MATHÉMATIQUES ONT INVESTI LA CRYPTOLOGIE (2)](https://images.math.cnrs.fr/Comment-les-Mathematiques-ont-investi-la-cryptologie-2.html)
* Cisco Networking Academy : [Initiation aux réseaux](http://cisco.ofppt.info/ccna1/index.html)
* Cisco Networking Academy : [Notions de base sur le routage et la commutation](http://cisco.ofppt.info/ccna2/index.html)
* Jeu en ligne [Cryptis](https://chiffrer.info/cryptris/)
* Wandida, APFL : Introduction à l'Informatique - Réseaux
    * [Introduction](https://youtu.be/VTgqWNcDNXQ)
    * [Messages et protocoles](https://youtu.be/WnWoy0Zv8Q8)
    * [Couches](https://youtu.be/HbtQaG1dnLg)
    * [Encapsulation](https://youtu.be/-ymIlUaqV58)
    * [Topologie](https://youtu.be/aIpa4jRwSqo)
    * [Commutation](https://youtu.be/TygPbu9dHgg)
    * [Routage](https://youtu.be/fKqepuQwYMQ)
    * [Calcul de routes](https://youtu.be/uj2ifTXkYo8)
    * [Protocoles](https://youtu.be/ZDPCK07vbSg)
    * [Types de réseaux](https://youtu.be/T9TRQTQZ0oU)
* Wandida, APFL : Introduction à l'Informatique - Sécurité
    * [Introduction](https://youtu.be/giR8p4vQkV0)
    * [Principes](https://youtu.be/kQPMwJyFScQ)
    * [Privacité](https://youtu.be/RB2jgbdbNTs)
    * [Confidentialité](https://youtu.be/AD3d-t4GlMA)
    * [Intégrité](https://youtu.be/BBYvWMj-gOU)
    * [Authentification](https://youtu.be/LVcJATu2l2Y)
    * [Biométrie et témoins](https://youtu.be/V8eA3y4tgV8)
    * [Identification](https://youtu.be/04pB8AyotMY)
    * [Utilisation](https://youtu.be/TSd9_WGazNk)